<a href="https://colab.research.google.com/github/AIManifest/deforum-stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $ {\large \textsf{AI Manifest's Deforum Stable Diffusion v0.7 Custom Notebook}}$

**I DO NOT CLAIM ANY OFFICIAL CODE IN THIS NOTEBOOK**

THIS NOTEBOOK IS PURELY FOR MY FUN AND ALL CREDITS ARE DUE
TO THEIR RESPECTIVE OWNERS. I SIMPLY MADE A FUN INTERFACE TO 
INTERACT WITH FOR STABLE DIFFUSION, UPSCALING, INTERPOLATION, AND MODEL MERGING.

[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). Notebook by [deforum](https://discord.gg/upmXXsrwZc)

[Quick Guide](https://docs.google.com/document/d/1RrQv7FntzOuLg4ohjRZPVL7iptIyBhwwbcEYEW2OfcI/edit?usp=sharing) to Deforum v0.7

In [ ]:
#@title $  {\large \textsf{Environment Setup / NVIDIA GPU -- **Mandatory** Run to Setup Everything for this Notebook}}$
print("\033[0;34m")
mount_drive_for_colab = True #@param {type:"boolean"}
if mount_drive_for_colab:
  from google.colab import drive
  drive.mount("/content/drive/")
import subprocess, os, sys
from tqdm.notebook import tqdm
from IPython import display
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"\033[0;34m{sub_p_res[:-1]}")
import subprocess, time, gc, os, sys
!pip install -qq -U kora
def setup_environment():
    start_time = time.time()
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("\033[0;34m..setting up environment")

        # weird hack
        import torch
        
        all_process = [
            ['pip', 'install', 'facexlib>=0.2.5', 'gfpgan>=1.3.5', 'basicsr>=1.4.2', 'sk-video>=1.1.10', 'omegaconf', 'einops==0.4.1', 'pytorch-lightning==1.7.7', 'torchmetrics', 'transformers', 'safetensors', 'kornia'],
            ['git', 'clone', 'https://github.com/AIManifest/deforum-stable-diffusion.git'],
            ['git', 'clone', 'https://github.com/AIManifest/Practical-RIFE.git'],
            ['git', 'clone', 'https://github.com/AIManifest/Real-ESRGAN.git'],
            ['git', 'clone', 'https://github.com/AIManifest/ESRGAN.git'],
            ['pip', 'install', '-qq', '--user', 'ffmpeg-python'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn','torchsde','open-clip-torch','numpngw', 'imageio-ffmpeg'],
        ]
        for process in tqdm(all_process):
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
        ])
        if use_xformers_for_colab:

            print("\033[0;34m..installing triton and xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20221202', 'xformers==0.0.16']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
    else:
        sys.path.extend([
            'src'
        ])
    end_time = time.time()
    print(f"\033[0;34m..environment set up in {end_time-start_time:.0f} seconds")
    return

setup_environment()
with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
  f.write('')
sys.path.extend([
  'deforum-stable-diffusion/',
  'deforum-stable-diffusion/src',
])
import torch
import cv2
import random
import clip
import yaml
from IPython.display import display
from ipywidgets import widgets
from IPython import display
import glob
from types import SimpleNamespace
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
from helpers.aesthetics import load_aesthetics_model




In [ ]:
#@title $ {\large \textsf{Path Setup}}$
print("\033[0;34m")
def Root():
    models_path = "models" #@param {type:"string"}
    configs_path = "configs" #@param {type:"string"}
    output_path = "outputs" #@param {type:"string"}
    mount_google_drive = True #@param {type:"boolean"}
    models_path_gdrive = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion" #@param {type:"string"}
    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}

    #@markdown **Model Setup**
    map_location = "cuda" #@param ["cpu", "cuda"]
    model_config = "custom" #@param ["custom","v2-inference.yaml","v2-inference-v.yaml","v1-inference.yaml"]
    model_checkpoint =  "custom" #@param ["custom","v2-1_768-ema-pruned.ckpt","v2-1_512-ema-pruned.ckpt","768-v-ema.ckpt","512-base-ema.ckpt","Protogen_V2.2.ckpt","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
    custom_config_path = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/Realistic_Dream_Nova_Deliberate.yaml" #@param {type:"string"}
    custom_checkpoint_path = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/Realistic_Dream_Nova_Deliberate.ckpt" #@param {type:"string"}
#@markdown $ \color{red} {\textsf{Leave These Blank if you Don't Need to Use Them, }}$
#@markdown 
#@markdown $ \color{red} {\textsf{But Generally, These Directories Should be Where All of }}$
#@markdown 
#@markdown $ \color{red} {\textsf{Your Embeddings, HyperNetworks, and Parent Directories are Respectively}}$
    embeddings_dir = "/content/drive/MyDrive/sd/stable-diffusion-webui/textual_inversion/2023-02-13/xxpjccrajararaxx/embeddings" #@param{type:'string'}
    hypernetwork_dir = "/content/drive/MyDrive/AI/hypernetworks" #@param{type:'string'}
    data_dir = "/content/drive/MyDrive/sd/stable-diffusion-webui" #@param{type:'string'}
    #@markdown **Don't use yet, still working on it**
    use_xformers = True #@param{type:'boolean'}
    use_sub_quad_attention = False #@param{type:'boolean'}
    use_split_attention_v1 = False #@param{type:'boolean'}
    use_split_cross_attention_forward_invokeAI = False #@param{type:'boolean'}
    use_cross_attention_attnblock_forward = False #@param{type:'boolean'}

    variables = locals()
    with open("Deforum.yaml", "w") as file:
      yaml.dump(variables, file)
    return variables

root = Root()
root = SimpleNamespace(**root)

root.models_path, root.output_path = get_model_output_paths(root)
root.model, root.device = load_model(root, load_on_run_all=True, check_sha256=True)

In [ ]:
#@title $ {\large \textsf{Animation Settings}}$
def DeforumAnimArgs():

    #@markdown ####**Animation:**
    animation_mode = 'None' #@param ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    max_frames = 6000 #@param {type:"number"}
    border = 'wrap' #@param ['wrap', 'replicate'] {type:'string'}

    #@markdown $\color{red} {\textsf {Experimental! Produces Very Nice Movements.. use rotation_3d_w}}$
    use_quaternion_rotation = True #@param {type:"boolean"}

    #@markdown ####**Motion Parameters:**
    angle = "0:(0)"#@param {type:"string"}
    zoom = "0:(1.02+0.02*sin(2*3.14*t/300))"#@param {type:"string"}
    translation_x = "0:(3*sin(2*3.14*t/300)-3)"#@param {type:"string"}
    translation_y = "0:(3+0.02*sin(2*3.14*t/300)-3)"#@param {type:"string"}
    translation_z = "0: (2), 30: ((-6*(sin(3.141*t/300)**50)+3) + (+0.1 *(sin(3.141*t/900)**50)+3))"#@param {type:"string"}
    rotation_3d_x = "0:(-0.2*sin(2*3.14*t/270))"#@param {type:"string"}
    rotation_3d_y = "0:(-0.2*sin(2*3.14*t/270))"#@param {type:"string"}
    rotation_3d_z = "0: (0.3*(cos(3.141*t/150)**900)+0.30)"#@param {type:"string"}
    rotation_3d_w = "0: (-0.3*(cos(3.141*t/150)**900)+0.30)" #@param {type:"string"}
    flip_2d_perspective = False #@param {type:"boolean"}
    perspective_flip_theta = "0:(0)"#@param {type:"string"}
    perspective_flip_phi = "0:(t%15)"#@param {type:"string"}
    perspective_flip_gamma = "0:(0)"#@param {type:"string"}
    perspective_flip_fv = "0:(53)"#@param {type:"string"}
    enable_cfg_scale_schedule = True #@param {type:"boolean"}
    cfg_scale_schedule = "0:(9), 30: (15*(cos(3.141*t/90)**20000)+9), 4026: (12), 4040: (15*(cos(3.141*t/90)**20000)+9)" #@param {type: "string"}
    noise_schedule = "0:(.02)"#@param {type:"string"}
    strength_schedule = "0:(.60), 30: (-.2*(cos(3.141*t/90)**20000)+0.57)"#@param {type:"string"}
    contrast_schedule = "0: (1.0)"#@param {type:"string"}
    hybrid_comp_alpha_schedule = "0:(1)" #@param {type:"string"}
    hybrid_comp_mask_blend_alpha_schedule = "0:(0.5)" #@param {type:"string"}
    hybrid_comp_mask_contrast_schedule = "0:(1)" #@param {type:"string"}
    hybrid_comp_mask_auto_contrast_cutoff_high_schedule =  "0:(100)" #@param {type:"string"}
    hybrid_comp_mask_auto_contrast_cutoff_low_schedule =  "0:(0)" #@param {type:"string"}

    # #@markdown ####**Checkpoint Scheduling:**
    # enable_checkpoint_scheduling = False#@param {type:"boolean"}
    # checkpoint_schedule = '0: ("model1.ckpt"), 100: ("model2.ckpt")'

    #@markdown ####**Cadence Scheduling:**
    enable_cadence_schedule = True #@param {type:"boolean"}
    cadence_schedule = "0:(4),300:(6), 600:(8), 900:(4), 1200:(4),1400:(6), 1600:(8), 1900:(4)" #@param{type:'string'}

    #@markdown ####**Sampler Scheduling:**
    enable_schedule_samplers = False #@param {type:"boolean"}
    sampler_schedule = "0:('euler'),10:('dpm2'),20:('dpm2_ancestral'),30:('heun'),40:('euler'),50:('euler_ancestral'),60:('dpm_fast'),70:('dpm_adaptive'),80:('dpmpp_2s_a'),90:('dpmpp_2m')" #@param {type:"string"}

    #@markdown ####**Unsharp mask (anti-blur) Parameters:**
    kernel_schedule = "0: (5)"#@param {type:"string"}
    sigma_schedule = "0: (1.0)"#@param {type:"string"}
    amount_schedule = "0: (0.2)"#@param {type:"string"}
    threshold_schedule = "0: (0.0)"#@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'Match Frame 0 RGB' #@param ['None', 'Match Frame 0 HSV', 'Match Frame 0 LAB', 'Match Frame 0 RGB', 'Video Input'] {type:'string'}
    color_coherence_video_every_N_frames = 1 #@param {type:"integer"}
    color_force_grayscale = False #@param {type:"boolean"}
    diffusion_cadence = '4' #@param ['1','2','3','4','5','6','7','8'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3#@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 40#@param {type:"number"}
    padding_mode = 'border'#@param ['border', 'reflection', 'zeros'] {type:'string'}
    sampling_mode = 'bicubic'#@param ['bicubic', 'bilinear', 'nearest'] {type:'string'}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4'#@param {type:"string"}
    extract_nth_frame = 1#@param {type:"number"}
    overwrite_extracted_frames = True #@param {type:"boolean"}
    use_mask_video = False #@param {type:"boolean"}
    video_mask_path ='/content/video_in.mp4'#@param {type:"string"}

    #@markdown ####**Hybrid Video for 2D/3D Animation Mode:**
    hybrid_generate_inputframes = False #@param {type:"boolean"}
    hybrid_use_first_frame_as_init_image = False #@param {type:"boolean"}
    hybrid_motion = "None" #@param ['None','Optical Flow','Perspective','Affine']
    hybrid_motion_use_prev_img = False #@param {type:"boolean"}
    hybrid_flow_method = "DIS Medium" #@param ['DenseRLOF','DIS Medium','Farneback','SF']
    hybrid_composite = False #@param {type:"boolean"}
    hybrid_comp_mask_type = "None" #@param ['None', 'Depth', 'Video Depth', 'Blend', 'Difference']
    hybrid_comp_mask_inverse = False #@param {type:"boolean"}
    hybrid_comp_mask_equalize = "None" #@param  ['None','Before','After','Both']
    hybrid_comp_mask_auto_contrast = False #@param {type:"boolean"}
    hybrid_comp_save_extra_frames = False #@param {type:"boolean"}
    hybrid_use_video_as_mse_image = False #@param {type:"boolean"}

    #@markdown ####**Interpolation:**
    interpolate_key_frames = False #@param {type:"boolean"}
    interpolate_x_frames = 4 #@param {type:"number"}
    
    #@markdown ####**Resume Animation:**
    resume_from_timestring = False #@param {type:"boolean"}
    resume_timestring = "20230213123159" #@param {type:"string"}

    return locals()

In [ ]:
#@title $ {\large \textsf{Simple Prompt Modifier Generator}}$
#@markdown $ {\large \textsf{Run the Cell to get popular artists, UHD words, art styles.}}$
#@markdown $ {\large \textsf{The list will be developed more as we go along..}}$
import random
from ipywidgets import widgets, Label
from IPython import display
#artists
psychedelic_artists = ["Pablo Amaringo", "Chuck Arnett", "Chris Dyer", "Doug Binder",  "Brummbaer", "Mark Boyle", "Joan Hills",]
surrealist_artists = ["Genevieve Leavold", "Sam Wilde", "Adam Lawrence", "Violet Polsangi",  "Kazuhiro Higashi", "Kim Marra", "Ricardo Harris-Fuentes",]
anime_artists = ["George Morikaw", "Keisuke Itagaki", "Yoichi Takahash", "Hirohiko Araki", "Masashi Kishimoto", "Yoshihiro Togashi", "Hajime Isayama", "Gosho Aoyama", "Akira Toriyama", "Eiichiro Oda",]

#high definition
high_definition = ["4K","High-resolution","High-quality","Sharp","Detailed","Vibrant","Clear","Realistic","Immersive","Stunning","Crisp","Bright","Colorful","Smooth","Life-like","Rich","Lifelike","Vivid","Gorgeous","Beautiful","Striking","Sharpness","Depth","Intense","Brilliant","Extraordinary","Dynamic","Elegant","Expansive","Fantastic","Fine","Outstanding","Magnificent","Mesmerizing","Incredible","Impressive","Sensational","Spectacular","Superb","Stupendous","Tremendous","Unbelievable","Unreal"]

#art style
art_style = ["Renaissance","Baroque","Rococo","Gothic","Impressionism","Post-Impressionism","Expressionism","Art Nouveau","Futurism","Cubism","Surrealism","Abstract Expressionism","Pop Art","Minimalism","Conceptual Art","Realism","Romanticism","Neoclassicism","Pre-Raphaelitism","Fauvism","Orphism","De Stijl","Suprematism","Constructivism","Dadaism","Expressionist Architecture","Bauhaus","Art Deco","Futurist Architecture","Concrete Art","Arte Povera","New Objectivity","Op Art","Hyperrealism","Photorealism","Magic Realism","Social Realism","Early Netherlandish Painting","High Renaissance","Mannerism","Northern Renaissance","Baroque Classicism","Caravaggism","Dutch Golden Age Painting","Romanticism in Art","American Regionalism","Abstract Illusionism","Abstract Impressionism","Neo-Expressionism","Hyperrealistic Sculpture","Environmental Art","New Media Art","Contemporary Realism","Neo-Pop Art"]

button1 = widgets.Button(description="Psychedelic Artists", button_style="danger")

def on_button1_clicked(b):

  selected_artists = random.sample(psychedelic_artists, 3)
  print("Psychedelic Artists: " + ", ".join(selected_artists))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button1.on_click(on_button1_clicked)

button2 = widgets.Button(description="Surrealist Artists", button_style="success")

def on_button2_clicked(b):

  selected_artists = random.sample(surrealist_artists, 3)
  print("Surrealist Artists: " + ", ".join(selected_artists))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button2.on_click(on_button2_clicked)

button3 = widgets.Button(description="Anime Artists", button_style="info")

def on_button3_clicked(b):

  selected_artists = random.sample(anime_artists, 3)
  print("Anime Artists: " + ", ".join(selected_artists))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button3.on_click(on_button3_clicked)

button_all = widgets.Button(description="Remix", button_style="Warning")

def on_button_all_clicked(b):

  first_artist = random.choice(psychedelic_artists)
  second_artist = random.choice(surrealist_artists)
  third_artist = random.choice(anime_artists)
  art_styles = random.choice(art_style)
  hd_word = random.choice(high_definition)
  print("Remix of Prompt Words: {}, {}, {}, {}, {}".format(first_artist, second_artist, third_artist, art_styles, hd_word))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button_all.on_click(on_button_all_clicked)

button4 = widgets.Button(description="High Definition Words", button_style="success")

def on_button4_clicked(b):

  selected_hd = random.sample(high_definition, 4)
  print("High Definition Words: " + ", ".join(selected_hd))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button4.on_click(on_button4_clicked)

button5 = widgets.Button(description="Art Styles", button_style="info")

def on_button5_clicked(b):

  selected_style = random.sample(art_style, 4)
  print("Art Styles: " + ", ".join(selected_style))
  # label = Label(value="Selected artists: " + ", ".join(selected_artists))
  # display(label)
button5.on_click(on_button5_clicked)

display.display(button1, button2, button3, button4, button5, button_all)

In [ ]:
# conditional (postitive) prompts
cond_prompts = {
0: "a phenomenal portrait of a feared being comprised of a population of microorganisms, cells, fear inoculum, fearful, UHD, very very detailed, clear, crisp, high resolution, best quality",

}

# unconditional (negative) prompts
uncond_prompts = {
    0: "streaks, artefacts, bad drawing, incoherent, messy, low resolution, pixelated, nude, nsfw, naked, low quality, multiple",
}

In [ ]:
#@title $ {\large \textsf{Load Settings}}$

import time
start_time = time.time()

override_settings_with_file = False #@param {type:"boolean"}
settings_file = "custom" #@param ["custom", "512x512_aesthetic_0.json","512x512_aesthetic_1.json","512x512_colormatch_0.json","512x512_colormatch_1.json","512x512_colormatch_2.json","512x512_colormatch_3.json"]
custom_settings_file = "/content/drive/MyDrive/Settings.txt"#@param {type:"string"}

def DeforumArgs():
    #@markdown **Image Settings**
    W = 512 #@param
    H = 512 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    bit_depth_output = 8 #@param [8, 16, 32] {type:"raw"}

    #@markdown **Sampling Settings**
    seed = -1 #@param
    sampler = 'dpm2_ancestral' #@param ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim", "dpm_fast", "dpm_adaptive", "dpmpp_2s_a", "dpmpp_2m"]
    steps = 120 #@param
    scale = 9 #@param
    ddim_eta = 0.0 #@param
    dynamic_threshold = None
    static_threshold = None   

    #@markdown **Save & Display Settings**
    save_samples = True #@param {type:"boolean"}
    save_settings = True #@param {type:"boolean"}
    display_samples = True #@param {type:"boolean"}
    save_sample_per_step = False #@param {type:"boolean"}
    show_sample_per_step = False #@param {type:"boolean"}

    #@markdown **Batch Settings**
    n_batch = 1 #@param
    n_samples = 6 #@param
    batch_name = "fear" #@param {type:"string"}
    filename_format = "{timestring}_{index}_{prompt}.png" #@param ["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png"]
    seed_behavior = "random" #@param ["iter","fixed","random","ladder","alternate"]
    seed_iter_N = 1 #@param {type:'integer'}
    make_grid = False #@param {type:"boolean"}
    grid_rows = 2 #@param 
    outdir = get_output_folder(root.output_path, batch_name)

    #@markdown **Init Settings**
    use_init = False #@param {type:"boolean"}
    strength = 0.65 #@param {type:"number"}
    strength_0_no_init = True # Set the strength to 0 automatically when no init image is used
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    add_init_noise = True #@param {type:"boolean"}
    init_noise = 0.015 #@param
    # Whiter areas of the mask are areas that change more
    use_mask = False #@param {type:"boolean"}
    use_alpha_as_mask = False # use the alpha channel of the init image as the mask
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" #@param {type:"string"}
    invert_mask = False #@param {type:"boolean"}
    # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
    mask_brightness_adjust = 1.0  #@param {type:"number"}
    mask_contrast_adjust = 1.0  #@param {type:"number"}
    # Overlay the masked image at the end of the generation so it does not get degraded by encoding and decoding
    overlay_mask = True  # {type:"boolean"}
    # Blur edges of final overlay mask, if used. Minimum = 0 (no blur)
    mask_overlay_blur = 5 # {type:"number"}

    #@markdown **Exposure/Contrast Conditional Settings**
    mean_scale = 0 #@param {type:"number"}
    var_scale = 0 #@param {type:"number"}
    exposure_scale = 0 #@param {type:"number"}
    exposure_target = 0.5 #@param {type:"number"}

    #@markdown **Color Match Conditional Settings**
    colormatch_scale = 0 #@param {type:"number"}
    colormatch_image = "https://www.saasdesign.io/wp-content/uploads/2021/02/palette-3-min-980x588.png" #@param {type:"string"}
    colormatch_n_colors = 4 #@param {type:"number"}
    ignore_sat_weight = 0 #@param {type:"number"}

    #@markdown **CLIP\Aesthetics Conditional Settings**
    clip_name = 'ViT-L/14' #@param ['ViT-L/14', 'ViT-L/14@336px', 'ViT-B/16', 'ViT-B/32']
    clip_scale = 0 #@param {type:"number"}
    aesthetics_scale = 0 #@param {type:"number"}
    cutn = 1 #@param {type:"number"}
    cut_pow = 0.0001 #@param {type:"number"}

    #@markdown **Other Conditional Settings**
    init_mse_scale = 0 #@param {type:"number"}
    init_mse_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    blue_scale = 0 #@param {type:"number"}
    
    #@markdown **Conditional Gradient Settings**
    gradient_wrt = 'x0_pred' #@param ["x", "x0_pred"]
    gradient_add_to = 'both' #@param ["cond", "uncond", "both"]
    decode_method = 'linear' #@param ["autoencoder","linear"]
    grad_threshold_type = 'dynamic' #@param ["dynamic", "static", "mean", "schedule"]
    clamp_grad_threshold = 0.2 #@param {type:"number"}
    clamp_start = 0.2 #@param
    clamp_stop = 0.01 #@param
    grad_inject_timing = list(range(1,10)) #@param

    #@markdown **Speed vs VRAM Settings**
    cond_uncond_sync = False #@param {type:"boolean"}
    precision = 'autocast' 
    C = 4
    f = 8

    cond_prompt = ""
    cond_prompts = ""
    uncond_prompt = ""
    uncond_prompts = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None
    seed_internal = 0

    return locals()

args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    load_args(args_dict, anim_args_dict, settings_file, custom_settings_file, verbose=False)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

# Load clip model if using clip guidance
if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
    root.clip_model = clip.load(args.clip_name, jit=False)[0].eval().requires_grad_(False).to(root.device)
    if (args.aesthetics_scale > 0):
        root.aesthetics_model = load_aesthetics_model(args, root)

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

with open("Deforum.yaml", "w") as file:
  yaml.dump(DeforumAnimArgs(), file)
  yaml.dump(DeforumArgs(), file)
  yaml.dump(cond_prompts, file)
  yaml.dump(uncond_prompts, file)

# dispatch to appropriate renderer
if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(root, anim_args, args, cond_prompts, uncond_prompts)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(root, anim_args, args, cond_prompts, uncond_prompts)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(root, anim_args, args, cond_prompts, uncond_prompts)
else:
    render_image_batch(root, args, anim_args, cond_prompts, uncond_prompts)

print("\033[0;34m..Your Animation Has Completed, Congratulations..")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

# $ {\large \textsf{Create Video From Frames}}$


In [ ]:
#@title $ { \textsf{COMPILE FRAMES TO VIDEO(SUBPROCESS--SOMETIMES FASTER)}}$
import time
import subprocess

start_time = time.time()
#@markdown **This cell will compile ALL images in the folder you provide, so make sure that the folder you specify has the correct images and do not have any unwanted images.**
vid_fps = 30 #@param {type:'number'}
image_ext = '*.png' #@param ['*.png', '*.PNG', '*.jpg', '*.jpeg', '*.JPEG', '*.JPG']
input_path = '/content/drive/MyDrive/AI/StableDiffusion/2023-02/xxpjccrajararaxx' # @param {type: 'string'}
#@markdown **The input folder with the frames you want to create a video with.**

raw_vid = '/content/temp.mp4' # @param {type: 'string'}
#@markdown **This `raw_vid` param results in an uncompressed, unformatted video, so to enable compatibility with most video players, we compress it to `compressed_vid`**

compressed_vid = '/content/drive/MyDrive/xxpjccrajararaxx.mp4' # @param {type: 'string'}
#@markdown **This `compressed_vid` is now compatible with most video players and should play properly.**
make_gif = False #@param {type:"boolean"}
#@markdown **Please ensure you change the extension of your video to `.gif` if you intend to make gifs**

%cd $input_path

process = subprocess.Popen(['ffmpeg', '-framerate', f"{vid_fps}", '-pattern_type', 'glob', '-i', image_ext, '-y', raw_vid], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

while True:
    line = process.stdout.readline()
    if not line:
        break
    print(line)

output, error = process.communicate()

display_ffmpeg_output=False #@param {type:"boolean"}

if display_ffmpeg_output:
  display.display(output)
  display.display(error)

%cd "/content/"
if not make_gif:
  process = subprocess.Popen(['ffmpeg', '-i', raw_vid, '-c:v', 'libx264', '-rc', 'vbr', '-cq', '19', '-pix_fmt', 'yuv420p', '-b:v', '0', '-y', compressed_vid], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  while True:
      line = process.stdout.readline()
      if not line:
          break
      print(line)

  output, error = process.communicate()

else:

  process = subprocess.Popen(['ffmpeg', '-i', raw_vid, '-r', str(vid_fps), '-y', compressed_vid], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  while True:
      line = process.stdout.readline()
      if not line:
          break
      print(line)

  output, error = process.communicate()

if display_ffmpeg_output:
  display.display(output)
  display.display(error)

elapsed_time = time.time() - start_time

print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $ { \textsf{COMPILE FRAMES TO VIDEO(FFMPEG)}}$
import time

start_time = time.time()

#@markdown **This cell will compile ALL images in the folder you provide, so make sure that the folder you specify has the correct images and do not have any unwanted images.**
vid_fps = 30 #@param {type:'number'}
image_ext = "'*.png'" #@param ['*.png', '*.PNG', '*.jpg', '*.jpeg', '*.JPEG', '*.JPG']
input_path = '/content/drive/MyDrive/AI/StableDiffusion/2023-02/NeonMutation_SeventhHell ' # @param {type: 'string'}
#@markdown **The input folder with the frames you want to create a video with.**
raw_vid = '/content/temp.mp4' # @param {type: 'string'}
#@markdown **This `raw_vid` param results in an uncompressed, unformatted video, so to enable compatibility with most video players, we compress it to `compressed_vid`**

compressed_vid = '/content/NeonMutation_SeventhHell.mp4' # @param {type: 'string'}
#@markdown **This `compressed_vid` is now compatible with most video players and should play properly.**

make_gif = False #@param {type:"boolean"}
#@markdown **Please ensure you change the extension of your video to `.gif` if you intend to make gifs**

%cd $input_path

!ffmpeg -framerate $vid_fps \
 -pattern_type glob \
 -i  $image_ext \
 -y $raw_vid

%cd '/content'
if not make_gif:
  !ffmpeg -i $raw_vid \
  -c:v libx264 \
  -rc vbr \
  -cq 19 \
  -pix_fmt yuv420p \
  -b:v 0 \
  -y $compressed_vid

else:
  !ffmpeg \
  -y \
  -i $raw_vid \
  -r $vid_fps \
  $compressed_vid

elapsed_time = time.time() - start_time

print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $ {\textsf{Display The Video Using kora(may error out, try uninstalling with "pip uninstall kora" if it doesn't work)!}}$
from kora.drive import upload_public
from IPython.display import HTML
url = '/content/238_RIFE.mp4' #@param{type:'string'}
url = upload_public(url)
# mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/kong/20230123004448.mp4"
# mp4 = open(mp4_path,'rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# then display it
HTML(f"""<video src={url} width=500 controls/>""")

# $ \color{cyan} {\large \textsf{Model Merger}}$

In [ ]:
#@title $ \color{cyan} {\large \textsf{Merge Model!}}$
import os
import re
import shutil

import torch
import tqdm
from types import SimpleNamespace
import safetensors.torch
import time
start_time = time.time()

#@markdown **It Appears That When Merging Safetensors, `map_location` must be `cpu`. Please ensure to follow this. You can also try `cuda` but it didn't work for me.**
def Load_checkpoints_to_be_merged():
  ckpt_dir = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion" #@param{type:'string'}
  primary_model_name = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/DeliberateDream.ckpt" #@param{type:'string'}
  secondary_model_name = "/content/Realistic_Vision_V1_3.ckpt" #@param{type:'string'}
  tertiary_model_name = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/Deliberate_Nova_Dream.ckpt" #@param{type:'string'}
  output_modelname = "Realistic_Dream_Nova_Deliberate" #@param{type:'string'}
  config_source = "/content/drive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/DeliberateDream.yaml" #@param{type:'string'}
  map_location = "cuda" #@param['cuda', 'cpu']
  interp_method = "Add difference" #@param["No interpolation", "Weighted sum", "Add difference"]
  multiplier = 0.7 #@param{type:'slider', min:0.1, max:1, step:0.1}
  save_as_half = False #@param{type:'boolean'}
  custom_name = "cuda" #@param['cuda', 'cpu']
  checkpoint_format = "ckpt" #@param['ckpt', 'safetensor']
  bake_in_vae = "/content/drive/MyDrive/AI/models/vae-ft-mse-840000-ema-pruned.ckpt" #@param{type:'string'}
  discard_weights = True #@param{type:'boolean'}
  return locals()

load_checkpoints_to_be_merged = Load_checkpoints_to_be_merged()
load_checkpoints_to_be_merged = SimpleNamespace(**load_checkpoints_to_be_merged)

chckpoint_dict_replacements = {
    'cond_stage_model.transformer.embeddings.': 'cond_stage_model.transformer.text_model.embeddings.',
    'cond_stage_model.transformer.encoder.': 'cond_stage_model.transformer.text_model.encoder.',
    'cond_stage_model.transformer.final_layer_norm.': 'cond_stage_model.transformer.text_model.final_layer_norm.',
}

def _load_vae_dict(model, vae_dict_1):
    model.first_stage_model.load_state_dict(vae_dict_1)
    model.first_stage_model.to("cuda")

def load_vae_dict(filename, map_location):
    vae_ckpt = read_state_dict(filename, map_location=map_location)
    vae_dict_1 = {k: v for k, v in vae_ckpt.items() if k[0:4] != "loss" and k not in vae_ignore_keys}
    return vae_dict_1


def load_vae(model, vae_file=None, vae_source="from unknown source"):
    global vae_dict, loaded_vae_file
    # save_settings = False


    if vae_file:
      _load_vae_dict(model, vae_file)
    else:
      assert os.path.isfile(vae_file), f"VAE {vae_source} doesn't exist: {vae_file}"
      print(f"Loading VAE weights {vae_source}: {vae_file}")
            

    vae_dict_1 = load_vae_dict(vae_file, load_checkpoints_to_be_merged.map_location)
    _load_vae_dict(model, vae_dict_1)

    vae_file = vae_dict_1.copy()

    loaded_vae_file = vae_file



def find_checkpoint_config(info):
    config = os.path.splitext(load_checkpoints_to_be_merged.primary_model_name) + ".yaml"
    if os.path.exists(config):
        return config

    return config

def transform_checkpoint_dict_key(k):
    for text, replacement in chckpoint_dict_replacements.items():
        if k.startswith(text):
            k = replacement + k[len(text):]

    return k

def get_state_dict_from_checkpoint(pl_sd):
    pl_sd = pl_sd.pop("state_dict", pl_sd)
    pl_sd.pop("state_dict", None)

    sd = {}
    for k, v in pl_sd.items():
        new_key = transform_checkpoint_dict_key(k)

        if new_key is not None:
            sd[new_key] = v

    pl_sd.clear()
    pl_sd.update(sd)

    return pl_sd

def read_state_dict(checkpoint_file, print_global_state=False, map_location=None):
    _, extension = os.path.splitext(checkpoint_file)
    if extension.lower() == ".safetensors":
        device = load_checkpoints_to_be_merged.map_location
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        pl_sd = safetensors.torch.load_file(checkpoint_file, device=device)
    else:
        pl_sd = torch.load(checkpoint_file, map_location=map_location)

    if print_global_state and "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")

    sd = get_state_dict_from_checkpoint(pl_sd)
    return sd

def create_config(ckpt_result, config_source, a, b, c):

    cfg = load_checkpoints_to_be_merged.config_source

    filename, _ = os.path.splitext(load_checkpoints_to_be_merged.output_modelname)
    checkpoint_filename = os.path.dirname(load_checkpoints_to_be_merged.primary_model_name) + "/" + ckpt_result + ".yaml"
    print("Copying config:")
    print("   from:", cfg)
    print("     to:", checkpoint_filename)
    shutil.copyfile(cfg, checkpoint_filename)


checkpoint_dict_skip_on_merge = ["cond_stage_model.transformer.text_model.embeddings.position_ids"]


def to_half(tensor, enable):
    if enable and tensor.dtype == torch.float:
        return tensor.half()

    return tensor

vae_ignore_keys = {"model_ema.decay", "model_ema.num_updates"}
vae_dict = {}
def run_modelmerger(primary_model_name, secondary_model_name, tertiary_model_name, interp_method, multiplier, save_as_half, custom_name, checkpoint_format, config_source, bake_in_vae, discard_weights):

    def fail(message):
        textinfo = message
        return message

    def weighted_sum(theta0, theta1, alpha):
        return ((1 - alpha) * theta0) + (alpha * theta1)

    def get_difference(theta1, theta2):
        return theta1 - theta2

    def add_difference(theta0, theta1_2_diff, alpha):
        return theta0 + (alpha * theta1_2_diff)

    def filename_weighted_sum():
        a = load_checkpoints_to_be_merged.primary_model_name
        b = load_checkpoints_to_be_merged.secondary_model_name
        Ma = round(1 - load_checkpoints_to_be_merged.multiplier, 2)
        Mb = round(load_checkpoints_to_be_merged.multiplier, 2)

        return f"{Ma}({a}) + {Mb}({b})"

    def filename_add_difference():
        a = load_checkpoints_to_be_merged.primary_model_name
        b = load_checkpoints_to_be_merged.secondary_model_name
        c = load_checkpoints_to_be_merged.tertiary_model_name
        M = round(load_checkpoints_to_be_merged.multiplier, 2)

        return f"{a} + {M}({b} - {c})"

    def filename_nothing():
        return load_checkpoints_to_be_merged.primary_model_name

    theta_funcs = {
        "Weighted sum": (filename_weighted_sum, None, weighted_sum),
        "Add difference": (filename_add_difference, get_difference, add_difference),
        "No interpolation": (filename_nothing, None, None),
    }
    filename_generator, theta_func1, theta_func2 = theta_funcs[load_checkpoints_to_be_merged.interp_method]
    job_count = (1 if theta_func1 else 0) + (1 if theta_func2 else 0)

    if not primary_model_name:
        return fail("Failed: Merging requires a primary model.")

    primary_model_info = load_checkpoints_to_be_merged.primary_model_name

    if theta_func2 and not secondary_model_name:
        return fail("Failed: Merging requires a secondary model.")

    secondary_model_info = load_checkpoints_to_be_merged.secondary_model_name if theta_func2 else None

    if theta_func1 and not tertiary_model_name:
        return fail(f"Failed: Interpolation method ({interp_method}) requires a tertiary model.")

    tertiary_model_info = load_checkpoints_to_be_merged.tertiary_model_name if theta_func1 else None

    result_is_inpainting_model = False

    if theta_func2:
        textinfo = f"Loading B"
        print(f"Loading {secondary_model_info}...")
        theta_1 = read_state_dict(secondary_model_info, map_location='cpu')
    else:
        theta_1 = None

    if theta_func1:
        textinfo = f"Loading C"
        print(f"Loading {tertiary_model_info}...")
        theta_2 = read_state_dict(tertiary_model_info, map_location='cpu')

        textinfo = 'Merging B and C'
        sampling_steps = len(theta_1.keys())
        for key in tqdm.tqdm(theta_1.keys()):
            if key in checkpoint_dict_skip_on_merge:
                continue

            if 'model' in key:
                if key in theta_2:
                    t2 = theta_2.get(key, torch.zeros_like(theta_1[key]))
                    theta_1[key] = theta_func1(theta_1[key], t2)
                else:
                    theta_1[key] = torch.zeros_like(theta_1[key])

            sampling_steps += 1
        del theta_2


    textinfo = f"Loading {primary_model_info}..."
    print(f"Loading {primary_model_info}...")
    theta_0 = read_state_dict(primary_model_info, map_location='cpu')

    print("Merging...")
    textinfo = 'Merging A and B'
    sampling_steps = len(theta_0.keys())
    for key in tqdm.tqdm(theta_0.keys()):
        if theta_1 and 'model' in key and key in theta_1:

            if key in checkpoint_dict_skip_on_merge:
                continue

            a = theta_0[key]
            b = theta_1[key]

            # this enables merging an inpainting model (A) with another one (B);
            # where normal model would have 4 channels, for latenst space, inpainting model would
            # have another 4 channels for unmasked picture's latent space, plus one channel for mask, for a total of 9
            if a.shape != b.shape and a.shape[0:1] + a.shape[2:] == b.shape[0:1] + b.shape[2:]:
                if a.shape[1] == 4 and b.shape[1] == 9:
                    raise RuntimeError("When merging inpainting model with a normal one, A must be the inpainting model.")

                assert a.shape[1] == 9 and b.shape[1] == 4, f"Bad dimensions for merged layer {key}: A={a.shape}, B={b.shape}"

                theta_0[key][:, 0:4, :, :] = theta_func2(a[:, 0:4, :, :], b, multiplier)
                result_is_inpainting_model = True
            else:
                theta_0[key] = theta_func2(a, b, multiplier)

            theta_0[key] = to_half(theta_0[key], save_as_half)

        sampling_steps += 1

    del theta_1

    bake_in_vae_filename = load_checkpoints_to_be_merged.bake_in_vae
    if bake_in_vae_filename is not None:
        print(f"Baking in VAE from {bake_in_vae_filename}")
        textinfo = 'Baking in VAE'
        vae_dict = load_vae_dict(bake_in_vae_filename, map_location='cpu')

        for key in vae_dict.keys():
            theta_0_key = 'first_stage_model.' + key
            if theta_0_key in theta_0:
                theta_0[theta_0_key] = to_half(vae_dict[key], save_as_half)

        del vae_dict

    if save_as_half and not theta_func2:
        for key in theta_0.keys():
            theta_0[key] = to_half(theta_0[key], save_as_half)

    # if discard_weights:
    #     regex = re.compile(discard_weights)
    #     for key in list(theta_0):
    #         if re.search(regex, key):
    #             theta_0.pop(key, None)

    ckpt_dir = load_checkpoints_to_be_merged.ckpt_dir
    filename = filename_generator() if custom_name == '' else custom_name
    filename += ".inpainting" if result_is_inpainting_model else ""
    filename += "." + checkpoint_format

    output_modelname = os.path.join(ckpt_dir, filename)

    textinfo = "Saving"
    print(f"Saving to {output_modelname}...")

    _, extension = os.path.splitext(output_modelname)
    if extension.lower() == ".safetensors":
        safetensors.torch.save_file(theta_0, output_modelname, metadata={"format": "pt"})
    else:
        torch.save(theta_0, output_modelname)


    ckpt_cfg = create_config(load_checkpoints_to_be_merged.output_modelname,
                  load_checkpoints_to_be_merged.config_source,
                  primary_model_info,
                  secondary_model_info,
                  tertiary_model_info)

    print(f"Checkpoint saved to {output_modelname}.")
    textinfo = "Checkpoint saved"
    

    return load_checkpoints_to_be_merged.output_modelname, ckpt_cfg

run_modelmerger(load_checkpoints_to_be_merged.primary_model_name,
                load_checkpoints_to_be_merged.secondary_model_name, 
                load_checkpoints_to_be_merged.tertiary_model_name, 
                load_checkpoints_to_be_merged.interp_method, 
                load_checkpoints_to_be_merged.multiplier, 
                load_checkpoints_to_be_merged.save_as_half, 
                load_checkpoints_to_be_merged.output_modelname, 
                load_checkpoints_to_be_merged.checkpoint_format, 
                load_checkpoints_to_be_merged.config_source, 
                load_checkpoints_to_be_merged.bake_in_vae, 
                load_checkpoints_to_be_merged.discard_weights)

print("\033[0;34m..Model Merging Has Completed, Congratulations..")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

# $\color{salmon} {\large \textsf{REAL-ESRGAN Video Upscaling!}}$

Please Be Sure to Run The Enviornment Setup Cell!


In [ ]:
import os
import cv2
import time

print("\033[0;34m")

start_time = time.time()

#@title $\color{salmon} {\large \textsf { 📹 RUN REAL-ESRGAN VIDEO UPSCALE PHASE (from video) 📹}}$ 
model_name = 'realesr-general-x4v3' #@param ['realesr-general-x4v3', 'realesr-general-wdn-x4v3', 'RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
lowres_vid = '/content/drive/MyDrive/parabola/parabola_p.mp4' #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input your lowres video path here to be upscaled}}$
highres_vid_dir = '/content/drive/MyDrive/parabola/' #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input your high res video output path here}}$
if not os.path.exists(highres_vid_dir):
  os.makedirs(highres_vid_dir)
outscale = "5" #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
denoising_strength = 1 #@param{type:'slider', step:0.01, min:0.01, max:1}
target_fps = 30 #@param{type:'number'}
fp32 = 'fp32'
gpu_id = 0 #@param ['0', '1', '2', '3', '4']
use_face_enhance = False #@param {type:"boolean"}
if not use_face_enhance:
  %cd '/content/Real-ESRGAN/'
  !python3 /content/Real-ESRGAN/inference_realesrgan_video.py \
  --model_name $model_name \
  --outscale $outscale \
  -dn $denoising_strength \
  --fp32 \
  --input $lowres_vid \
  --fps $target_fps \
  --output $highres_vid_dir

else:
  %cd '/content/Real-ESRGAN/'
  !python3 /content/Real-ESRGAN/inference_realesrgan_video.py \
  --model_name $model_name \
  --outscale $outscale \
  --face_enhance \
  -dn $denoising_strength \
  --fp32 \
  --input $lowres_vid \
  --fps $target_fps \
  --output $highres_vid_dir

%cd '/content/'
print("\033[0;34m...Upscaling Phase Has Completed, You May Proceed...")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
import os
import cv2
import time
print("\033[0;34m")
start_time = time.time()
#@title $\color{salmon} {\large \textsf { 📹 !BATCH! RUN REAL-ESRGAN VIDEO UPSCALE PHASE (from video) 📹}}$ 
model_name = 'realesr-general-x4v3' #@param ['realesr-general-x4v3', 'realesr-general-wdn-x4v3', 'RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
batch_viddir = "/content/drive/MyDrive/LINUM/lowres/lowres_input_video/" #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input folder path with videos to upscale **WILL UPSCALE ALL VIDEOS IN THE FOLDER**}}$
highres_vid_dir = '/content/drive/MyDrive/RIFE_Batch' #@param{type:'string'}
#@markdown $\color{salmon} {\textsf {input your high res video output path here}}$
if not os.path.exists(highres_vid_dir):
  os.makedirs(highres_vid_dir)
outscale = "4" #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
denoising_strength = 1 #@param{type:'slider', step:0.01, min:0.01, max:1}
target_fps = 30 #@param{type:'number'}
fp32 = 'fp32'
gpu_id = 0 #@param ['0', '1', '2', '3', '4']
updir = os.path.split(highres_vid_dir)[0] + "/"

for vid in os.listdir(batch_viddir):
  new_vid = os.path.join(batch_viddir, vid)
  print(new_vid)
  %cd '/content/Real-ESRGAN'
  !python3 /content/Real-ESRGAN/inference_realesrgan_video.py --model_name $model_name --outscale $outscale -dn $denoising_strength --fp32 --input $new_vid --fps $target_fps --output $highres_vid_dir
  !mv $new_vid $updir --verbose
  print(f"\033[93mMoving {new_vid} to {updir}.")
%cd '/content/'
print("\033[0;34m...Upscaling Phase Has Completed, You May Proceed...")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

# $\color{orange} {\large \textsf{RIFE 4.6 Interpolation!}}$

Please Be Sure to Run The Enviornment Setup Cell!

Added RIFE 4.5 and 4.4 Model. Standard Model is 4.6.

These are just options, you can use the 4.6 model as it is the latest version.

In [ ]:
#@title $\color{orange} {\large \textsf{Define Params and Run RIFE!}}$
import time
print("\033[0;34m")
start_time = time.time()
model_dir = "RIFE46" #@param['RIFE46', 'RIFE45', 'RIFE44']
# model_dir = "/content/Practical-RIFE/" + model_dir
input_RIFE = '/content/drive/MyDrive/parabola/parabola_p_out.mp4' # @param {type: 'string'}
#@markdown $\color{orange} {\textsf{THIS IS THE FILE NAME FOR YOUR COMPRESSED VID}}$

output_RIFE = '/content/drive/MyDrive/parabola/parabola_p_out_RIFE.mp4' # @param {type: 'string'}
#@markdown $\color{orange} {\textsf{INPUT YOUR OUTPUT NAME FOR THE INTERPOLATED VIDEO}}$
fps_RIFE = 60 # @param {type: 'number'}
target_length_RIFE = 694 # @param {type: 'number'}
target_scale_RIFE = 0.5 #@param ['0.25', '0.5', '1.0', '2.0', '4.0']
_skip_RIFEsmoothing = False
#@markdown $\color{orange} {\textsf{Additional Params}}$
print("\n ------------------------------------\n")
print("\n Beginning RIFE motion smoothing phase... \n")

%cd /content/Practical-RIFE/

class Detection:
  #def __init__(self):
  #  pass
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = float(int(a_string))
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(measured_duration,target_length_RIFE): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = fps_RIFE * target_length_RIFE
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x

#----------------------------
_import_mp4_file = False

if _import_mp4_file:
  measured_fps = Detection.detect_fps(input_RIFE)
  print("\n NOTICE: Detected average FPS of ",input_RIFE," is ",measured_fps)
  measured_duration = Detection.detect_duration(input_RIFE)
else: 
  measured_fps = Detection.detect_fps(input_RIFE)
  print("\n NOTICE: Detected average FPS of ",{input_RIFE}," is ",measured_fps)
  measured_duration = Detection.detect_duration(input_RIFE)

print("\n NOTICE: Detected duration INTEGER (in seconds) is ",measured_duration)

if measured_duration < 1: #failsafe
  print("\n NOTICE: Your input appears to be less than one second... \n")
  measured_duration = 1

exp_value = Detection.exp_calc(measured_duration,target_length_RIFE)


print("\n NOTICE: Target duration currently rounds to the closest --exp RIFE can handle. \n")

if (exp_value < 0.5):
  _skip_RIFEsmoothing = False
  print("\n NOTICE: Your fps_RIFE doesn't necessitate RIFE motion smoothing. Skipping RIFE...\n")

if _skip_RIFEsmoothing:
  print("\n NOTICE: Skipping RIFE motion smoothing...\n")
else:
  #---RUN RIFE------------------------------------
  print("\n NOTICE: Running RIFE... \n")
  %cd /content/Practical-RIFE/
  !python3 /content/Practical-RIFE/inference_video.py \
  --model=$model_dir \
  --fps=$fps_RIFE \
  --exp=$exp_value \
  --video=$input_RIFE \
  --scale=$target_scale_RIFE \
  --output=$output_RIFE
  #--exp=$exp_value
  #---END--------------------------------------
#!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname
#END OF MOTION SMOOTHING PHASE
print("\n \033[0;34mEnd of RIFE interpolation phase.\n")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $\color{orange} {\large \textsf{!BATCH! Define Params and Run RIFE!}}$
import time
start_time = time.time()
print("\033[0;34m")
%cd /content/Practical-RIFE/

class Detection:
  #def __init__(self):
  #  pass
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = float(int(a_string))
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(measured_duration,target_length_RIFE): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = fps_RIFE * target_length_RIFE
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x

#----------------------------
model_dir = "RIFE46" #@param['RIFE46', 'RIFE45', 'RIFE44']
rife_viddir = "/content/drive/MyDrive/RIFE_Batch" #@param{type:'string'}
#@markdown $\color{orange} {\textsf{THIS IS THE PATH TO THE FOLDER WITH YOUR VIDEOS TO BE INTERPOLATED}}$
for rife_vid in os.listdir(rife_viddir):
    new_vid = os.path.join(rife_viddir, rife_vid)
    print("\033[35mNOTICE: Interpolation will be performed on: \n\033[0;34m", new_vid)
fps_RIFE = 60 # @param {type: 'number'}
# target_length_RIFE = 35 # @param {type: 'number'}
target_scale_RIFE = 0.5 #@param ['0.25', '0.5', '1.0', '2.0', '4.0']
_skip_RIFEsmoothing = False
#@markdown $\color{orange} {\textsf{Additional Params}}$
print("\n ------------------------------------\n")
print("\n \033[0;34mBeginning RIFE motion smoothing phase... \n")

#----------------------------

if _skip_RIFEsmoothing:
  print("\n \033[33mNOTICE: Skipping RIFE motion smoothing...\n")
else:
  #---RUN RIFE------------------------------------
  print("\n ------------------------------------\n")
  print("\n \033[33mNOTICE: Running RIFE... \n")
  for rife_vid in os.listdir(rife_viddir):
    new_vid = os.path.join(rife_viddir, rife_vid)
    output_RIFE = f"{os.path.splitext(new_vid)[0]}_RIFE{os.path.splitext(new_vid)[1]}"
    measured_fps = Detection.detect_fps(new_vid)
    print("\n \033[33mNOTICE: Detected average FPS of ",new_vid," is ",measured_fps)
    measured_duration = Detection.detect_duration(new_vid)

    print("\n \033[33mNOTICE: Detected duration INTEGER (in seconds) is ",measured_duration)

    if measured_duration < 1: #failsafe
      print("\n \033[33mNOTICE: Your input appears to be less than one second... \n")
      measured_duration = 1
    target_length_RIFE = measured_duration
    exp_value = Detection.exp_calc(measured_duration,target_length_RIFE)

    print("\n \033[33mNOTICE: Target duration currently rounds to the closest --exp RIFE can handle. \n")

    if (exp_value < 0.5):
      _skip_RIFEsmoothing = False
      print("\n \033[33mNOTICE: Your fps_RIFE doesn't necessitate RIFE motion smoothing. Skipping RIFE...\n")

    if _skip_RIFEsmoothing:
      print("\n \033[33mNOTICE: Skipping RIFE motion smoothing...\n")
    print(f"\033[0;34mSaving Video to: ", output_RIFE)
    print("\n ------------------------------------\n")
    %cd /content/Practical-RIFE/
    !python3 /content/Practical-RIFE/inference_video.py \
    --model=$model_dir \
    --fps=$fps_RIFE \
    --exp=$exp_value \
    --video=$new_vid \
    --scale=$target_scale_RIFE \
    --output=$output_RIFE
  
%cd "/content/"
  #---END--------------------------------------
#!ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $_constant_quality -qmin $_constant_quality -qmax $_constant_quality -b:v 0 $output_fullpathname
#END OF MOTION SMOOTHING PHASE
print("\n \033[0;34mEnd of RIFE interpolation phase.\n")
elapsed_time = time.time() - start_time
print("\033[0;34mElapsed time:", elapsed_time, "seconds")

In [ ]:
#@title $\color{orange} {\large \textsf {Analyze Lowres Vid Dimensions and FPS}}$ 
print("\033[0;34m")
vcap = cv2.VideoCapture(output_RIFE)

width = vcap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT )
fps =  vcap.get(cv2.CAP_PROP_FPS)
num_frames = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vcap.get(cv2.CAP_PROP_FPS)

# calculate the video length in seconds
video_length = num_frames / fps
# get the size of the file in bytes
file_size_bytes = os.path.getsize(output_RIFE)

# convert the file size to megabytes
file_size_mb = file_size_bytes / (1024 * 1024)

# print the file size in megabytes
print(f"File size: {file_size_mb:.2f} MB")
# print the video length
print(f"Video length: {video_length} seconds")
print(f"Width is: {width}.")
print(f"Height is: {height}.")
print(f"FPS is: {fps}.")

# $ \color{violet} {\large \textsf{REAL-ESRGAN Single Image or Frame Folder Upscaling!}}$


In [ ]:
#@title $ {\large \textsf { 🚀 RUN REAL-ESRGAN UPSCALE PHASE (from frames) 🚀}}$
#@markdown $ {\large \textsf {Input_dir is either a single image or a folder of images}}$
model_name = 'realesr-general-x4v3' #@param ['realesr-general-x4v3', 'realesr-general-wdn-x4v3', 'RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus'] {type:"string"}
input_dir = '/content/drive/MyDrive/AI/StableDiffusion/2023-02/fear/20230220142306_00004_a_phenomenal_portrait_of_a_feared_being_comprised_of_a_population_of_microorganisms_cells_fear_inoculum_fearful_UHD_very_very_detailed_clear_crisp_high_resoluti.png' #@param{type:'string'}
output_dir = '/content/drive/MyDrive' #@param{type:'string'}
outscale = "6" #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
denoising_strength = 1 #@param{type:'slider', step:0.01, min:0.01, max:1}
fp32 = 'fp32'
gpu_id = 0 #@param ['0', '1', '2', '3', '4']
%cd '/content/'
%cd '/content/Real-ESRGAN/'
!python3 /content/Real-ESRGAN/inference_realesrgan.py --model_name $model_name --outscale $outscale -dn $denoising_strength --fp32 --gpu-id $gpu_id --input $input_dir --output $output_dir
%cd '/content/'
print("\033[0;34m...Upscaling Phase Has Completed, You May Proceed...")

In [ ]:
from PIL import Image
display_the_image = True #@param{type:'boolean'}
display_image = "/content/drive/MyDrive/20230220142306_00004_a_phenomenal_portrait_of_a_feared_being_comprised_of_a_population_of_microorganisms_cells_fear_inoculum_fearful_UHD_very_very_detailed_clear_crisp_high_resoluti_out.png" #@param{type:'string'}
#@markdown Path to the Image you want to Display
if display_the_image:
    im = Image.open(display_image)
    display.display(im)

def analyze_image_dimensions(file_path):
    with Image.open(file_path) as img:
        width, height = img.size
        print(f"Image Width: \033[0;34m{width}, \033[0mImage Height: \033[0;34m{height}")

analyze_image_dimensions(display_image)

# $ \color{darkblue} {\large \textsf{ESRGAN Upscaling Interpolation}}$

(Use Multiple Models with Trained ESRGAN Models)

This Implementation offers the ability to use different

ESRGAN models to achieve the upscaling result you may desire

and otherwise not get using a single upscaler.

In [ ]:
#@title $ {\large \textsf { 🚀 RUN ESRGAN UPSCALE INTERPOLATION (from frames), Using One or More ESRGAN Models! 🚀}}$
#@markdown **`esrgan_input` may be a single image or a folder path to your images:**

#@markdown **This is a rather advanced implementation, Follow this structure:**

#@markdown $ {\color{yellow} {\textbf{model_path@amount_to_interpolate_by|model_path@amount_to_interpolate_by}}}$

#@markdown $ {\color{yellow} { \textbf {Example: 4x_FatalPixels_340000_G.pth@50|4x_fatal_Anime_500000_G.pth@50}}}$

#@markdown **Interpolation may not work with each model due to their structure(SRVGGNET, RRDB, ETC). Mix and match to your preference. Additionally, you can choose from the massive selection here at [UpscaleWiki](https://upscale.wiki/wiki/Model_Database)**
!pip install -qq rich
print("\033[0;34m")
use_upscale_interpolation = True #@param {type:"boolean"}
esrgan_input = "/content/7ED86DAC-7064-41B3-A757-8EAB600D23E9.jpeg" #@param {type:"string"}
if not os.path.exists(esrgan_input):
  os.makedirs(esrgan_input)
esrgan_output = "/content/ESRGAN/output" #@param {type:"string"}
if not os.path.exists(esrgan_output):
  os.makedirs(esrgan_output)

esrgan_model_1 = "4x_fatal_Anime_500000_G.pth" #@param ['4x_Ultrasharp.pth', 'RRDB_ESRGAN_x4.pth', '4x_FatalPixels_340000_G.pth', '1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth', '4x_fatal_Anime_500000_G.pth', '4x-UltraSharp.pth', '4x_NMKD-YandereNeo-Lite_320k.pth', 'RRDB_PSNR_x4.pth']
esrgan_model_1_visibility = 80 #@param {type:"slider", min:5, max:100, step:5}
esrgan_model_2 = "4x-UltraSharp.pth" #@param ['4x_Ultrasharp.pth', 'RRDB_ESRGAN_x4.pth', '4x_FatalPixels_340000_G.pth', '1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth', '4x_fatal_Anime_500000_G.pth', '4x-UltraSharp.pth', '4x_NMKD-YandereNeo-Lite_320k.pth', 'RRDB_PSNR_x4.pth']
esrgan_model_2_visibility = 20 #@param {type:"slider", min:5, max:100, step:5}

if esrgan_model_1_visibility + esrgan_model_2_visibility != 100:
    print("\033[0;31mERROR: The sum of model visibility percentages must be 100.\033[0m")

if use_upscale_interpolation:
  %cd "/content/ESRGAN"
  !python upscale.py \
  "$esrgan_model_1@$esrgan_model_1_visibility|$esrgan_model_2@$esrgan_model_2_visibility" \
  --input $esrgan_input \
  --output $esrgan_output

else:
  %cd "/content/ESRGAN"
  !python upscale.py \
  "$esrgan_model_1" \
  --input $esrgan_input \
  --output $esrgan_output

%cd "/content/"

# $ {\large \textsf{Disconnect Runtime}}$

In [ ]:
skip_disconnect_for_run_all = False #@param {type: 'boolean'}

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()